<a href="https://colab.research.google.com/github/MrymHkmbdi/TFIDF_COSINESIMILARITY_INFORMATIONRETRIEVAL/blob/main/IRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Library Installation and Setup

In [ ]:
pip install clean-text

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

## Phase 1 Libariries


In [ ]:
import re
import nltk
import pandas as pd
from cleantext import clean
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


## Phase 2 Libariries

In [ ]:
from pathlib import Path
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# Phase 1

#### in this phase the data read from plot_summaries.txt were converted to csv and cleaned.
#### After that feature of searching were added. in this part when you search a term, number o repeats and ids of movies containing the term will be shown.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/plot_summaries.txt', delimiter='\t', names=['id', 'overview'])
# dataframe = df.to_csv('plot.csv', index=False)

In [ ]:
ps = PorterStemmer()
cleans_data=[]
for i in range(0,len(data)):
  clean_data=clean(data.loc[i,"overview"],
        fix_unicode=True,               # fix various unicode errors
        to_ascii=True,                  # transliterate to closest ASCII representation
        lower=True,                     # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_phone_numbers=True,         # replace all phone numbers with a special token
        no_numbers=False,               # replace all numbers with a special token
        no_digits=False,                # replace all digits with a special token
        no_currency_symbols=True,      # replace all currency symbols with a special token
        no_punct=True,                 # remove punctuations
        replace_with_punct="",          # instead of removing punctuations you may replace them
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUMBER>",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        lang="en"                       # set to 'de' for German special handling
    )
  nltk_english_stopwords = stopwords.words('english')
  clean_data_final = ""
  words=re.sub("[^\w]", " ", clean_data).split()
  for w in words:
    if w not in nltk_english_stopwords:
      clean_data_final = clean_data_final + " "+ ps.stem(w)
  data.loc[i,"cleaned_data"] = clean_data_final

NameError: ignored

In [ ]:
data.to_csv('plot_cleaned.csv', index=False)

In [ ]:
cleaned_data_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/plot_cleaned.csv',encoding='utf-8')
ps = PorterStemmer()
cleans_data=[]
terms = []
counter = 0
input_term = input()
clean_data=clean('{}'.format(input_term),
        fix_unicode=True,               # fix various unicode errors
        to_ascii=True,                  # transliterate to closest ASCII representation
        lower=True,                     # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_phone_numbers=True,         # replace all phone numbers with a special token
        no_numbers=False,               # replace all numbers with a special token
        no_digits=False,                # replace all digits with a special token
        no_currency_symbols=True,      # replace all currency symbols with a special token
        no_punct=True,                 # remove punctuations
        replace_with_punct="",          # instead of removing punctuations you may replace them
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUMBER>",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        lang="en"                       # set to 'de' for German special handling
    )
ans = ps.stem(clean_data)
for i in range(0,len(cleaned_data_file)):
  for word in cleaned_data_file.loc[i, "cleaned_data"].split():
    if ans in word:
      terms.append(cleaned_data_file.loc[i, "id"])
    else:
      continue
mylist = list(dict.fromkeys(terms))
print('Term {} repeated {} times in films listed below: '.format(input_term, len(mylist)))
print(mylist)


In [ ]:
cleaned_data_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/plot_cleaned.csv',encoding='utf-8')
inv_idx_dict = {}
for i in range(0, len(clened_data_file)):
  for word in cleaned_data_file.loc[i, "cleaned_data"].split():
    if word not in inv_idx_dict.keys():
      inv_idx_dict[word] = [cleaned_data_file.loc[i, "id"]]
    else:
      inv_idx_dict[word].append(cleaned_data_file.loc[i, "id"])
for each_key, value in inv_idx_dict.items():
   inv_idx_dict[each_key] = list(set(value))
print(inv_idx_dict)

In [ ]:
(pd.DataFrame.from_dict(data=inv_idx_dict, orient='index')
   .to_csv('/content/drive/MyDrive/Colab Notebooks/inverted-index.csv', header=False))

# TF-IDF AND COSINE SIMILARITY

#### in this phase calculating the cosine similarity between whole document and a query was added. the result was a list of 10 most related movies to our query.

In [ ]:
plot_cleaned_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/irm-hw1/plot_cleaned.csv',encoding='utf-8')

In [ ]:
query = input()
ps = PorterStemmer()
cleans_data=[]
clean_data=clean('{}'.format(query),
        fix_unicode=True,               # fix various unicode errors
        to_ascii=True,                  # transliterate to closest ASCII representation
        lower=True,                     # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_phone_numbers=True,         # replace all phone numbers with a special token
        no_numbers=False,               # replace all numbers with a special token
        no_digits=False,                # replace all digits with a special token
        no_currency_symbols=True,      # replace all currency symbols with a special token
        no_punct=True,                 # remove punctuations
        replace_with_punct="",          # instead of removing punctuations you may replace them
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUMBER>",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        lang="en"                       # set to 'de' for German special handling
  )
nltk_english_stopwords = stopwords.words('english')
cleaned_query = ""
words=re.sub("[^\w]", " ", clean_data).split()
for w in words:
  if w not in nltk_english_stopwords:
    cleaned_query = cleaned_query + " "+ ps.stem(w)
print(cleaned_query)

In [ ]:
tfidf_score = TfidfVectorizer()
movie_tfidf = tfidf_score.fit_transform(plot_cleaned_file['cleaned_data'].to_list())
query_tfidf = tfidf_score.transform([cleaned_query])
similarity = cosine_similarity(query_tfidf, movie_tfidf)
for i, row in enumerate(similarity):
    plot_cleaned_file.loc[(-row).argsort()[:10]].to_csv("queryandmovies/{}.csv".format(query),index=False)

In [ ]:
queriy_and_movies = pd.read_csv("/content/drive/MyDrive/queryandmovies/1.csv")

In [ ]:
ids = []
overview = []
for i in range(len(queriy_and_movies)):
  overview.append(queriy_and_movies.loc[i, 'overview'])
  ids.append(queriy_and_movies.loc[i, 'id'])
zip_it = zip(ids, overview)
id_plot_dict = dict(zip_it)
for k,v in id_plot_dict.items():
  print('{}: {}\n'.format(k,v))

In [ ]:
tfidf_score = TfidfVectorizer()
plot = [plot_cleaned_file.loc[1, 'cleaned_data']]
tf_idf =  tfidf_score.fit_transform(plot)
df = pd.DataFrame(tf_idf.toarray(), columns=tfidf_score.get_feature_names())
df.to_csv('tfidf_table.csv')